In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

/home/pavithran/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pavithran/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pavithran/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pavithran/.local/lib/python3.7/site-packages/tensorboar

ModuleNotFoundError: No module named 'cv2'

In [2]:
def read_images(dataset_path):
    global classes
    im_path,labels=list(),list()
    classes=os.listdir(dataset_path)
    for i in classes:
        path=os.path.join(dataset_path,i)
        for j in os.listdir(path):
            path1=os.path.join(path,j)
            im_path.append(path1)
            lab=classes.index(i)
            labels.append(lab)
    for i in range(len(classes)):
        print('{}:{}'.format(classes[i],i))
    im_path=tf.convert_to_tensor(im_path)
    labels=tf.convert_to_tensor(labels)
    image,labels=tf.train.slice_input_producer([im_path,labels],shuffle=True)
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.resize_images(image, [32, 32])
    # Normalize
    image = image * 1.0/127.5 - 1.0
    X, Y = tf.train.batch([image, labels], batch_size=32,capacity=32 * 8,num_threads=4)
    return X,Y

In [3]:
dataset_path=''
dataset_path='/home/cocoslabs/human_pose/ssd/gester_dataset/'
X,Y=read_images(dataset_path)

sos:0
normal:1
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions fo

In [130]:
def conv2d(x,input,output,filter=3,strides=1,padding='SAME',activation='relu',name=None):
    w=tf.Variable(tf.random_normal([filter,filter,input,output]))
    b=tf.Variable(tf.random_normal([output]))
    x=tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding=padding)
    x=tf.nn.bias_add(x,b)
    if activation is 'relu':
        x=tf.nn.relu(x,name=name)
    elif(activation is 'tanh'):
        x=tf.nn.tanh(x,name=name)
    elif(activation is 'softmax'):
        x=tf.nn.softmax(x,name=name)
    else:
        print('give relu,tanh or softmax')
    return x


def maxpool2d(x, k=2,name=None):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME',name=name)

def averagepool2d(x,k=2,name=None):
    return tf.nn.avg_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME',name=name)

def dense(x,input,output,activation='tanh',name=None):
    w=tf.Variable(tf.random_normal([input,output]))
  
    b=tf.Variable(tf.random_normal([output]))
    
    x=tf.add(tf.matmul(x,w),b)
    
    if(activation is 'tanh'):
        x=tf.nn.tanh(x,name=name)
    elif(activation is 'softmax'):
        x=tf.nn.softmax(x,name=name)
    else:
        print('give tanh or softmax')
    return x
    

def lenet(x):    
    conv1=conv2d(x,1,6,filter=5,activation='tanh',padding='VALID',name='Conv1')
    print(conv1)
    conv1=averagepool2d(conv1,name='Avgpool1')
    print(conv1)
    conv2=conv2d(conv1,6,16,filter=5,activation='tanh',padding='VALID',name='Conv2')
    print(conv2)
    conv2=averagepool2d(conv2,name='Avgpool1')
    print('111111',conv2.shape)
    flatten=tf.reshape(conv2,[conv2.get_shape()[0],conv2.get_shape()[1]*conv2.get_shape()[2]*
                              conv2.get_shape()[3]],name='flatten')
    print(conv2)
    print(flatten)
    
    
    d1=dense(flatten,input=flatten.get_shape().as_list()[1],output=120,activation='tanh',name='Dense1')
    print(d1)
    d2=dense(d1,input=120,output=84,activation='tanh',name='Dense2')
    print(d2)
    model=dense(d2,input=84,output=2,activation='softmax',name='Dense3')
    print(model)

In [131]:
x=tf.placeholder(tf.float32,[None,32,32,1])

In [132]:
init=tf.global_variables_initializer()
model=lenet(X)
with tf.Session() as sess:
    coord=tf.train.Coordinator()
    tf.train.start_queue_runners()
    sess.run(init)
    sess.run(model)
    coord.request_stop()
    


Tensor("Conv1_34:0", shape=(32, 28, 28, 6), dtype=float32)
Tensor("Avgpool1_68:0", shape=(32, 14, 14, 6), dtype=float32)
Tensor("Conv2_34:0", shape=(32, 10, 10, 16), dtype=float32)
('111111', TensorShape([Dimension(32), Dimension(5), Dimension(5), Dimension(16)]))
Tensor("Avgpool1_69:0", shape=(32, 5, 5, 16), dtype=float32)
Tensor("flatten_27:0", shape=(32, 400), dtype=float32)
Tensor("Dense1_3:0", shape=(32, 120), dtype=float32)
Tensor("Dense2_3:0", shape=(32, 84), dtype=float32)
Tensor("Dense3_3:0", shape=(32, 2), dtype=float32)
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[{{node input_producer/input_producer/input_producer_EnqueueMany}}]]
ERROR:tensorflow:Exception in QueueRunner: Run call was cancelled
ERROR:tensorflow:Exception in QueueRunner: Run call was cancelled
ERROR:tensorflow:Exception in QueueRunner: Run call was cancelled
ERROR:tensorflow:Exception in QueueRunner: Run call was cancelled


Exception in thread QueueRunnerThread-input_producer/input_producer-input_producer/input_producer/input_producer_EnqueueMany:
Traceback (most recent call last):
  File "/home/cocoslabs/anaconda3/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/cocoslabs/anaconda3/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/cocoslabs/anaconda3/lib/python2.7/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/home/cocoslabs/anaconda3/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/home/cocoslabs/anaconda3/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
CancelledError: Enqueue operation was cancelled
	 [[{{node input_producer/input_producer/input_pro

TypeError: Fetch argument None has invalid type <type 'NoneType'>

Exception in thread QueueRunnerThread-batch/fifo_queue-batch/fifo_queue_enqueue:
Traceback (most recent call last):
  File "/home/cocoslabs/anaconda3/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/cocoslabs/anaconda3/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/cocoslabs/anaconda3/lib/python2.7/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/home/cocoslabs/anaconda3/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/home/cocoslabs/anaconda3/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
CancelledError: Run call was cancelled

